In this notebook we will test the candidate generation module 

In [1]:
# IMPORTs
from utils.task import Task
import json
import os
from src.pipeline.candidate_generation import candidate_generation
from utils.database_utils.db_info import get_db_schema
from dotenv import load_dotenv
from utils.prompt import load_prompt
import tiktoken
from tqdm import tqdm

load_dotenv()

True

In [2]:
# Function to load JSON data
def load_json_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    return data


# Function to create task object
def create_task(example):
    return Task(example)

In [3]:
# load the task data
filepath = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/subsampled_test.json"
data = load_json_data(filepath)
# load the retrieved entities
filepath_entities = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/retrieved_entities.json"
retrieved_entities = load_json_data(filepath_entities)
# load the retrieved context
filepath_context = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/retrieved_context.json"
retrieved_context = load_json_data(filepath_context)
# load the selected schema
filepath_schema = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/selected_schema.json"
selected_schema = load_json_data(filepath_schema)

In [4]:
# test candidate generation module with a random sample
index = 1
example = data[index]
task = create_task(example)

model = ("gpt-4")
ans = candidate_generation(task=task, retrieved_entities=retrieved_entities[index],
                           retrieved_context=retrieved_context[index], selected_schema=selected_schema[index],
                           model=model,
                           num_samples=1)
ans

{'chain_of_thought_reasoning': "To solve this query, I need to identify a school that meets several criteria: it must be a high school (EILCode = 'HS'), located in Merced county, provides Lunch Provision 2, and spans grades 9 to 12. First, I will filter schools based on these criteria. Since the database schema specifics are not provided, I assume there is a table for schools where I can find columns for EILCode, county, lunch provision, lowest grade, and highest grade. After filtering the schools based on these criteria, I will select the city location of the school. The query will involve selecting the city column from the schools table after applying the necessary WHERE conditions to filter by EILCode, county, lunch provision, and grade range. I will ensure to handle any potential NULL values in the columns used for filtering to avoid incorrect results.",
 'SQL': "SELECT T1.city FROM Schools T1 WHERE T1.EILCode = 'HS' AND T1.county = 'Merced' AND T1.lunchProvision = 2 AND T1.lowestG

In [5]:
#test candidate generation module with the subsampled test
model = "gpt-4"
num_samples = 1
res = []
for index, sample in tqdm(enumerate(data)):
    task = create_task(sample)
    ans = candidate_generation(task=task, retrieved_entities=retrieved_entities[index],
                               retrieved_context=retrieved_context[index], selected_schema=selected_schema[index],
                               model=model,
                               num_samples=num_samples)
    res.append(ans)
    print(str(index) + " : ", ans)

1it [00:11, 11.65s/it]

0 :  {'chain_of_thought_reasoning': "To answer the question, we need to identify the atom id associated with the molecule 'TR346' and count the distinct types of bonds that can be created by this molecule. First, we need to find the table that links molecules to their atom ids, which is likely a molecule table and an atom table. We assume there's a relationship table that connects molecules to atoms, possibly named 'molecule_atoms'. Next, we need to find the table that contains information about bonds, which might be named 'bonds' or similar, and it should relate to atoms. We will join these tables based on molecule id and atom id. We will filter the molecule by its name 'TR346', count the distinct bond types, and list the atom ids. The SQL query will involve selecting the atom id from the joined table of molecules and atoms, and a subquery or a separate count on the bond types, grouped by the molecule id.", 'SQL': "SELECT T1.atom_id, COUNT(DISTINCT T3.bond_type) AS bond_type_count FRO

2it [00:20,  9.88s/it]

1 :  {'chain_of_thought_reasoning': "To answer the question, I need to identify a school that meets several criteria: it must be a high school (EILCode = 'HS'), located in Merced county, with Lunch Provision 2, and it should cover grades from 9 to 12. First, I will filter the schools based on these criteria. Since the question specifically asks for the city location, I will select only the city column from the relevant table. I will use aliases for clarity in the query and ensure that the columns used for filtering (like grades and EILCode) do not contain null values to avoid incorrect data filtering.", 'SQL': "SELECT T1.city FROM Schools AS T1 WHERE T1.EILCode = 'HS' AND T1.county = 'Merced' AND T1.LunchProvision = 2 AND T1.lowestGrade = 9 AND T1.highestGrade = 12"}


3it [00:29,  9.46s/it]

...

12 :  {'chain_of_thought_reasoning': "To solve the query, we need to calculate the percentage of bonds that are double bonds (' = ') among all bonds associated with the molecule 'TR047'. First, we identify the relevant tables and columns: molecule_id and bond_type from a bonds table. We filter the bonds by molecule_id = 'TR047' and then calculate the percentage of those bonds that are double bonds. The percentage calculation involves counting the total number of bonds for 'TR047' and the number of those that are double bonds, then dividing the count of double bonds by the total bond count and multiplying by 100 to get the percentage. We use a CASE statement to conditionally count bonds where bond_type equals ' = '. The result is grouped by molecule_id to ensure the calculation is specific to 'TR047'.", 'SQL': "SELECT (SUM(CASE WHEN bond_type = ' = ' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) AS percent FROM bonds WHERE molecule_id = 'TR047'"}


14it [02:11,  9.73s/it]

13 :  {'chain_of_thought_reasoning': "To answer the question, I need to find the user who added a bounty of 50 to a post with a title mentioning 'variance'. First, I need to identify the relevant tables and columns. The 'BountyAmount' suggests a table related to bounties, and 'DisplayName' suggests a table related to users. The post title mentioning 'variance' implies a need to filter records based on the title of a post. I will start by identifying the tables: one for bounties, one for posts, and one for users. I will join these tables: bounties to posts (to link bounty to the specific post) and posts to users (to identify the user who made the post). The condition will be set on 'BountyAmount' to be 50 and the post title to include the word 'variance'. I will use LIKE '%variance%' for the title filter. The final output should only include the 'DisplayName' of the user, as per the instructions to select the name when the user is referred to. I will use aliases for clarity in the query

15it [02:20,  9.35s/it]

14 :  {'chain_of_thought_reasoning': "To answer the question, I need to find the dates when users from Rochester, NY obtained their badges. This involves two main steps: 1) Identifying users from Rochester, NY, which means I need to join the 'users' table with the 'badges' table since the location information is likely in the 'users' table and the badge obtainment dates are in the 'badges' table. 2) Filtering these users and retrieving the dates they obtained badges. I will use SQL JOIN to combine these tables on the user ID, filter by the location 'Rochester, NY', and select the badge date. I will ensure to handle any NULL values in the columns used for filtering and sorting.", 'SQL': "SELECT T2.Date FROM Users T1 JOIN Badges T2 ON T1.UserId = T2.UserId WHERE T1.Location = 'Rochester, NY' AND T2.Date IS NOT NULL"}


In [6]:
from pathlib import Path

schema_path = Path('C:/Users/yousf/Bureau/ConvergenceAI/CHESS_Impl/data/test/generated_candidate.json')
with open(schema_path, 'w') as f:
    json.dump(res, f, indent=4)

## Cost Estimation per task

In [5]:

PROMPT_PATH = os.getenv("PROMPT_ROOT_PATH") + "\\candidate_generation.txt"
prompt = load_prompt(PROMPT_PATH)

In [6]:

def tokens_calc(example):
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(example))
    return num_tokens

In [7]:
#prompt template tokens 
tokens_calc(prompt)

631

The prompt template has 631 tokens in total, and we have also 3 other variables (Database_Schema,Question and Hint).
After i see a formatted prompt example it contains about 2000 tokens (because database_schema will be long)
So let's suppose that input tokens is 2000

In [8]:
## Output tokens estimation from an example 
output_example = """
{'chain_of_thought_reasoning': "First, I identified the relevant columns from the question: CDSCode, County Code, School Type, Low Grade, High Grade, and County Name. Then, I determined that the question is asking for the city location of a high school level school with Lunch Provision 2, whose lowest grade is 9 and the highest grade is 12 in the county of Merced. I used the hint that high school can be represented as EILCode = 'HS'. I joined the frpm and schools tables based on the CDSCode, and then filtered the results to match the conditions specified in the question. Finally, I selected the City column from the schools table, which is the column that provides the city location.",
 'SQL': "SELECT City FROM schools WHERE EILCode = 'HS' AND County = 'Merced' AND LowGrade = '9' AND HighGrade = '12' AND NSLPProvisionStatus = 'Lunch Provision 2' AND CDSCode IN (SELECT CDSCode FROM frpm WHERE CountyCode = '02');"}
"""
tokens_calc(output_example)

227

Let's suppose the output tokens is 250

In [9]:
## Price calculation (just with gpt4 because in this module we don't use gpt3.5) 
input_price_per_token_gpt4 = 0.01 / 1000
output_price_per_token_gpt4 = 0.03 / 1000
price_gpt4 = 2000 * input_price_per_token_gpt4 + 250 * output_price_per_token_gpt4
print("estimated price per Task (GPT-4):", price_gpt4, "$")

estimated price per Task (GPT-4): 0.0275 $


In [10]:
## in this module there is a number of retrials so let's estimate the price with different number of retrials 

num_retrials = [1, 2, 3, 4, 5]
for num_retrial in num_retrials:
    total_price = price_gpt4 * num_retrial
    print("The estimated price with number of retrials of " + str(num_retrial) + ": ", total_price)

The estimated price with number of retrials of 1:  0.0275
The estimated price with number of retrials of 2:  0.055
The estimated price with number of retrials of 3:  0.0825
The estimated price with number of retrials of 4:  0.11
The estimated price with number of retrials of 5:  0.1375
